In [13]:
import numpy as np
import h5py
import json

from datetime import datetime
from copy import deepcopy
from pathlib import Path
from sklearn.preprocessing import StandardScaler
import tensorflow.keras as ks

from matbench.bench import MatbenchBenchmark
from crystalgnns.crystal_preprocessing.crystal_preprocessors import (RadiusAsymmetricUnitCell,
                                                                     KNNAsymmetricUnitCell,
                                                                     VoronoiAsymmetricUnitCell)
from crystalgnns.graph_dataset_tools.graph_tuple import HDFGraphTuple, GraphTuple
from crystalgnns.graph_dataset_tools.model_preprocessors import GNPreprocessor
from crystalgnns.datasets.matbench_datasets import MatbenchDataset
from crystalgnns.models.graph_network.graph_network import get_model

## Functions for Training/Evaluation Procedure

Define helper functions:

In [ ]:
def get_model_preprocessor(use_multiplicity = True, use_voronoi_area = False,
                           use_line_graph = True, line_graph_direction=[1,1]):
    model_inputs = [GNPreprocessor.Inputs.offset,
        GNPreprocessor.Inputs.atomic_number,
        GNPreprocessor.Inputs.edge_indices]
    if use_multiplicity:
        model_inputs.append(GNPreprocessor.Inputs.multiplicity)
    if use_voronoi_area:
        model_inputs.append(GNPreprocessor.Inputs.voronoi_ridge_area)
    if use_line_graph:
        model_inputs.append(GNPreprocessor.Inputs.line_graph_edge_indices)
    model_preprocessor = GNPreprocessor(inputs=model_inputs,
                                        line_graph_directions=line_graph_direction)
    return model_preprocessor

def get_lr_scheduler(dataset_size, batch_size, epochs, lr_start=0.0005, lr_stop=1e-5):
    steps_per_epoch = dataset_size / batch_size
    num_steps = epochs * steps_per_epoch
    scheduler = ks.optimizers.schedules.PolynomialDecay(initial_learning_rate=lr_start,
                                                        decay_steps=num_steps,
                                                        end_learning_rate=lr_stop)
    return scheduler

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

Define training procedure:

In [ ]:
def train_procedure(model_cfg, crystal_preprocessor, matbench_datasets_subset, use_multiplicity = True,
                    use_voronoi_area = False, use_line_graph = True, line_graph_direction = [1,1],
                    dataset_cache=Path('./dataset_cache'), results_cache=Path('./results'),
                    use_scaler=True, epochs=800, batch_size=64):
    # MatBench training procedure
    dataset_cache = Path(dataset_cache)
    results_cache = Path(results_cache)
    
    model_preprocessor = get_model_preprocessor(use_multiplicity=use_multiplicity,
                                                use_voronoi_area=use_voronoi_area,
                                                use_line_graph=use_line_graph,
                                                line_graph_direction=line_graph_direction)

    mb_dataset_cache = MatbenchDataset(dataset_cache)
    
    mb = MatbenchBenchmark(subset=matbench_datasets_subset, autoload=False)
    for idx_task, task in enumerate(mb.tasks):

        # Create/Access cached file for preprocessed crystals
        preprocessed_crystals_file = mb_dataset_cache.get_dataset_file(task.dataset_name, crystal_preprocessor)
        f = h5py.File(preprocessed_crystals_file, 'r')
        preprocessed_crystals = HDFGraphTuple(f)

        # Mapping from unique crystal ids in dataset to index of the cached file
        id_index_mapping = {id_.decode():i for i, id_ in enumerate(preprocessed_crystals.graph_attributes['dataset_id'][:])}
        def get_graphs(inputs):
            idxs = [id_index_mapping[id_] for id_ in inputs.index]
            return preprocessed_crystals[idxs]

        task.load()
        for fold in task.folds:
            intermediate_results_ = results_cache / task.dataset_name / str(fold)
            if intermediate_results_.exists():
                print('Load intermediate results')
                predictions = np.load(str(intermediate_results_ / 'predictions.npy'))
                history_dict = json.load(open(intermediate_results_ / 'history.json', 'r'))
            else:
                # Get training data with target values
                train_inputs, train_outputs = task.get_train_and_val_data(fold)

                train_graphs = get_graphs(train_inputs)
                train_graphs.graph_attributes['label'] = train_outputs.to_numpy()
                train_graphs.graph_attribute_names.append('label')
                x_train, y_train = model_preprocessor.to_ragged_tensors(train_graphs)
                y_train = np.expand_dims(y_train,-1)

                if use_scaler and task.metadata["task_type"] != "classification":
                    scaler = StandardScaler()
                    y_train = scaler.fit_transform(y_train)
                
                if task.metadata["task_type"] == "classification":
                    loss = ks.losses.BinaryCrossentropy(from_logits=True)
                    metrics = [ks.metrics.AUC(from_logits=True)]
                else:
                    loss = ks.losses.MeanAbsoluteError()
                    metrics = [ks.losses.MeanAbsoluteError(), ks.losses.MeanSquaredError()]
                    
                model = get_model(model_cfg.input_block_cfg,
                     [model_cfg.processing_block_cfg]* model_cfg.depth,
                     model_cfg.output_block_cfg,
                     multiplicity=use_multiplicity,
                     voronoi_ridge_area=use_voronoi_area,
                     line_graph=use_line_graph)
                scheduler = get_lr_scheduler(y_train.shape[0], batch_size, epochs,
                                             lr_start=0.0005, lr_stop=1e-5)
                optimizer = ks.optimizers.Adam(learning_rate=scheduler)
                model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
                
                # Train model
                start = datetime.now()
                history = model.fit(x_train, y_train,
                            batch_size=batch_size,
                            epochs=epochs)
                duration_in_seconds = (datetime.now() - start).total_seconds()

                # Get test data
                test_inputs = task.get_test_data(fold, include_target=False)
                test_graphs = get_graphs(test_inputs)
                test_graphs.graph_attributes['label'] = np.zeros(len(test_inputs)) # dummy values
                test_graphs.graph_attribute_names.append('label')
                x_test, _ = model_preprocessor.to_ragged_tensors(test_graphs)

                # Predict
                predictions = model.predict(x_test)
                
                if use_scaler and task.metadata["task_type"] != "classification":
                    predictions = scaler.inverse_transform(predictions)
                if predictions.shape[-1] == 1:
                    predictions = np.squeeze(predictions, axis=-1)

                intermediate_results_.mkdir(parents=True)
                history_dict = {k: np.array(v).tolist() for k,v in history.history.items()}
                history_dict['training_time'] = duration_in_seconds
                np.save(str(intermediate_results_ / 'predictions.npy'), predictions)
                json.dump(history_dict, open(intermediate_results_ / 'history.json', 'w'))
                model.save_weights(str(intermediate_results_ / 'weights.h5'))

            task.record(fold, predictions, params={
                'input_block': model_cfg.input_block_cfg,
                'processing_blocks': [model_cfg.processing_block_cfg]* model_cfg.depth,
                'output_block': model_cfg.output_block_cfg})
        f.close()
    return mb

## Model Configuration

Define configuration manually:

In [ ]:
units = 128
depth = 5

input_block_cfg = {'node_size': units,
 'edge_size': units,
 'atomic_mass': True,
 'atomic_radius': True,
 'electronegativity': True,
 'ionization_energy': True,
 'oxidation_states': True,
 'edge_embedding_args': {'bins_distance': 32,
  'max_distance': 8.0,
  'distance_log_base': 1.0,
  'bins_voronoi_area': None,
  'max_voronoi_area': None}}

processing_block_cfg = {'edge_mlp': {'units': [units]*5,
  'activation': ['swish']*5},
 'node_mlp': {'units': [units]*1,
  'activation': ['swish']*1},
 'global_mlp': None,
 'nested_blocks_cfgs': None,
 'aggregate_edges_local': 'sum',
 'aggregate_edges_global': None,
 'aggregate_nodes': None,
 'return_updated_edges': False,
 'return_updated_nodes': True,
 'return_updated_globals': True,
 'edge_attention_mlp_local': {'units': [32, 1],
  'activation': ['swish', 'swish']},
 'edge_attention_mlp_global': {'units': [32, 1],
  'activation': ['swish', 'swish']},
 'node_attention_mlp': {'units': [32, 1], 'activation': ['swish', 'swish']},
 'edge_gate': None,
 'node_gate': None,
 'global_gate': None,
 'residual_node_update': True,
 'residual_edge_update': False,
 'residual_global_update': False,
 'update_edges_input': [True, True, True, False],
 'update_nodes_input': [True, False, False],
 'update_global_input': [False, True, False],
 'multiplicity_readout': False}

output_block_cfg = {'edge_mlp': None,
 'node_mlp': None,
 'global_mlp': {'units': [1],
  'activation': ['linear']},
 'nested_blocks_cfgs': None,
 'aggregate_edges_local': 'sum',
 'aggregate_edges_global': None,
 'aggregate_nodes': 'mean',
 'return_updated_edges': False,
 'return_updated_nodes': True,
 'return_updated_globals': True,
 'edge_attention_mlp_local': {'units': [32, 1],
  'activation': ['swish', 'swish']},
 'edge_attention_mlp_global': {'units': [32, 1],
  'activation': ['swish', 'swish']},
 'node_attention_mlp': {'units': [32, 1], 'activation': ['swish', 'swish']},
 'edge_gate': None,
 'node_gate': None,
 'global_gate': None,
 'residual_node_update': False,
 'residual_edge_update': False,
 'residual_global_update': False,
 'update_edges_input': [True, True, True, False],
 'update_nodes_input': [True, False, False],
 'update_global_input': [False, True, False],
 'multiplicity_readout': True}

model_cfg = {
    'units': units,
    'depth': depth,
    'input_block_cfg': input_block_cfg,
    'processing_block_cfg': processing_block_cfg,
    'output_block_cfg': output_block_cfg
}

# Make keys accessible via dot notation
model_cfg = dotdict(model_cfg)

... or load from seperate python file/module:

In [ ]:
import coGN_config as model_cfg
# import coNGN_config as model_cfg

## Run Evaluation Procedure

In [ ]:
matbench_datasets_subset = ["matbench_dielectric", "matbench_phonons", "matbench_jdft2d"]
mb = train_procedure(model_cfg, KNNAsymmetricUnitCell(24),
                         matbench_datasets_subset,
                         use_multiplicity = True,
                         use_voronoi_area = False, use_line_graph = False, line_graph_direction = [1,1],
                         dataset_cache=Path('./dataset_cache'),
                         results_cache=Path('./results'),
                         use_scaler=True,
                         epochs=8,
                         batch_size=64)